In [1]:
#table extraction from hotel check-in documents using Amazon textract

import boto3
import csv
from pathlib import Path
from PyPDF2 import PdfReader, PdfWriter
import os

textractor = boto3.client("textract", region_name="ap-southeast-1")

file_path = Path(r"/Users/isaaclim/Desktop/OCR/mentis-ocr/cth2.pdf")

if not file_path.exists():
    raise FileNotFoundError(f"The file {file_path} does not exist.")

def split_pdf(file_path):
    reader = PdfReader(str(file_path))
    output_paths = []
    for i in range(len(reader.pages)):
        writer = PdfWriter()
        writer.add_page(reader.pages[i])
        output_path = file_path.with_name(f"{file_path.stem}_page_{i+1}.pdf")
        with open(output_path, "wb") as output_pdf:
            writer.write(output_pdf)
        output_paths.append(output_path)
    return output_paths

page_paths = split_pdf(file_path)

print(page_paths)

def extract_tables_from_page(page_path):
    with open(page_path, "rb") as document:
        imageBytes = bytearray(document.read())

    response = textractor.analyze_document(
        Document={"Bytes": imageBytes}, FeatureTypes=["TABLES"]
    )

    blocks = response["Blocks"]
    tables = []

    for block in blocks:
        if block["BlockType"] == "TABLE":
            table = []
            cell_map = {}
            for relationship in block.get("Relationships", []):
                if relationship["Type"] == "CHILD":
                    for id in relationship["Ids"]:
                        cell_map[id] = None

            for cell in blocks:
                if cell["BlockType"] == "CELL" and cell["Id"] in cell_map:
                    row_index = cell["RowIndex"]
                    col_index = cell["ColumnIndex"]
                    text = ""
                    for relationship in cell.get("Relationships", []):
                        if relationship["Type"] == "CHILD":
                            for id in relationship["Ids"]:
                                for item in blocks:
                                    if item["Id"] == id and item["BlockType"] == "WORD":
                                        text += item["Text"] + " "
                    text = text.strip()
                    cell_map[cell["Id"]] = (row_index, col_index, text)
                    table.append((row_index, col_index, text))

            tables.append(table)
    return tables

csv_file_path = Path(r"D:\Engineer\Aws - sagemaker\sagemaker-mob\extracted_table_3.csv")

with open(csv_file_path, mode="w", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    for page_path in page_paths:
        tables = extract_tables_from_page(page_path)
        for table in tables:
            max_row = max(cell[0] for cell in table)
            max_col = max(cell[1] for cell in table)
            table_data = [["" for _ in range(max_col)] for _ in range(max_row)]

            for cell in table:
                row_index, col_index, text = cell
                table_data[row_index - 1][col_index - 1] = text

            writer.writerows(table_data)
            writer.writerow([]) 

print(f"Output written to {csv_file_path}")

for i in page_paths:
    os.remove(Path(i))

[PosixPath('/Users/isaaclim/Desktop/OCR/mentis-ocr/cth2_page_1.pdf'), PosixPath('/Users/isaaclim/Desktop/OCR/mentis-ocr/cth2_page_2.pdf'), PosixPath('/Users/isaaclim/Desktop/OCR/mentis-ocr/cth2_page_3.pdf'), PosixPath('/Users/isaaclim/Desktop/OCR/mentis-ocr/cth2_page_4.pdf')]
Output written to D:\Engineer\Aws - sagemaker\sagemaker-mob\extracted_table_3.csv


In [3]:
import json
import logging
import os
from dotenv import load_dotenv

load_dotenv()

import boto3
from botocore.exceptions import ClientError

logger = logging.getLogger(__name__)

client = boto3.client(
    "bedrock-runtime",
    aws_access_key_id=os.getenv("aws_access_key_id"),
    aws_secret_access_key=os.getenv("aws_secret_access_key"),
    region_name="ap-southeast-2",
)

#/Users/isaaclim/Downloads/bedrocktest.py
import textwrap
from string import Template

insight_summarize = Template(
    textwrap.dedent(
        """
**Personality**
You are a personal assistant to a hotel concierge, your job is to determine whether the given dataframes carry the descriptions of
different relevant topics which include: "ROOM CATEGORY", "MEAL", "TRANSPORT", "EVENT". This can be determined by simply viewing the column
titles in each dataframe given to you. If the dataframe columns don't include any of the  above, simply output the integer 0.

**Table prompt**
$insight

**Input description**:
You will receive a list of strings consisting of the column titles of a dataframe,  \
You might receive input in languages other than English.

**Step description**:
IMPORTANT: Explicitly output a single integer that best describes the dataframe's contents based on its column titles: 

1 if "ROOM CATEGORY" is the subject of the dataframe
2 if "MEAL" is the subject of the dataframe
3 if "TRANSPORT" is the subject of the dataframe
4 if "EVENT" is the subject of the dataframe

if none of these words can be found within the given dataframe column titles, output 0.

**Output description**:
The output should strictly only be the following integers: 0, 1, 2, 3, 4
"""
    )
)

model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

In [4]:
#LLM integration to categorize each extracted table in preparation for final templating.

import csv
import os
from pathlib import Path
from PyPDF2 import PdfReader, PdfWriter
import boto3
from datetime import datetime
import pandas as pd

relevant = {}

# Initialize AWS Textract client
textractor = boto3.client("textract", region_name="ap-southeast-1")

# Define the PDF file path
file_path = Path(r"/Users/isaaclim/Desktop/OCR/mentis-ocr/cth2.pdf")
#file_path = Path(r"/Users/isaaclim/Desktop/OCR/mentis-ocr/Contract 3.pdf")


if not file_path.exists():
    raise FileNotFoundError(f"The file {file_path} does not exist.")

# Function to split PDF into individual pages
def split_pdf(file_path):
    reader = PdfReader(str(file_path))
    output_paths = []
    for i in range(len(reader.pages)):
        writer = PdfWriter()
        writer.add_page(reader.pages[i])
        output_path = file_path.with_name(f"{file_path.stem}_page_{i+1}.pdf")
        with open(output_path, "wb") as output_pdf:
            writer.write(output_pdf)
        output_paths.append(output_path)
    return output_paths

page_paths = split_pdf(file_path)

print(page_paths)

# Function to extract tables from each PDF page
def extract_tables_from_page(page_path):
    with open(page_path, "rb") as document:
        imageBytes = bytearray(document.read())

    response = textractor.analyze_document(
        Document={"Bytes": imageBytes}, FeatureTypes=["TABLES"]
    )

    blocks = response["Blocks"]
    tables = []

    for block in blocks:
        if block["BlockType"] == "TABLE":
            table = []
            cell_map = {}
            for relationship in block.get("Relationships", []):
                if relationship["Type"] == "CHILD":
                    for id in relationship["Ids"]:
                        cell_map[id] = None

            for cell in blocks:
                if cell["BlockType"] == "CELL" and cell["Id"] in cell_map:
                    row_index = cell["RowIndex"]
                    col_index = cell["ColumnIndex"]
                    text = ""
                    for relationship in cell.get("Relationships", []):
                        if relationship["Type"] == "CHILD":
                            for id in relationship["Ids"]:
                                for item in blocks:
                                    if item["Id"] == id and item["BlockType"] == "WORD":
                                        text += item["Text"] + " "
                    text = text.strip()
                    cell_map[cell["Id"]] = (row_index, col_index, text)
                    table.append((row_index, col_index, text))

            tables.append(table)
    return tables

# Counter for CSV file names
table_counter = 1

# Extract tables and write each table to a different CSV file
for page_path in page_paths:
    tables = extract_tables_from_page(page_path)
    for table in tables:
        max_row = max(cell[0] for cell in table)
        max_col = max(cell[1] for cell in table)
        table_data = [["" for _ in range(max_col)] for _ in range(max_row)]

        for cell in table:
            row_index, col_index, text = cell
            table_data[row_index - 1][col_index - 1] = text

        # Define the CSV file path dynamically
        csv_file_path = file_path.with_name(f"extracted_table_{table_counter}.csv")
        
        # Write the table data to the CSV file
        with open(csv_file_path, mode="w", newline="", encoding="utf-8") as csv_file:
            writer = csv.writer(csv_file)
            writer.writerows(table_data)

        print(f"Output written to {csv_file_path}")
        
        df_curr = pd.read_csv(csv_file_path)
        column_list = df_curr.columns
            
        # Increment the table counter
        table_counter += 1

        request_body = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 2048,
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": insight_summarize.substitute(insight=column_list),
                        },
                        {
                            "type": "text",
                            "text": "What's the category of the contents within this dataframe based on its given column titles?",
                        },
                    ],
                }
            ],
        }

        
        try:
            response = client.invoke_model(
                modelId=model_id,
                body=json.dumps(request_body),
            )
        
            # Process and print the response
            result = json.loads(response.get("body").read())
            input_tokens = result["usage"]["input_tokens"]
            output_tokens = result["usage"]["output_tokens"]
            output_list = result.get("content", [])
        
            print("Invocation details:")
            print(f"- The input length is {input_tokens} tokens.")
            print(f"- The output length is {output_tokens} tokens.")
        
            print(f"- The model returned {len(output_list)} response(s):")
            for output in output_list:
                print(output["text"])
        except ClientError as err:
            logger.error(
                "Couldn't invoke Claude 3 Sonnet. Here's why: %s: %s",
                err.response["Error"]["Code"],
                err.response["Error"]["Message"],
            )

        csv_file_path = str(csv_file_path)
        
        if (output["text"] != "0") & (len(output["text"]) == 1):
            relevant[csv_file_path] = output["text"]
            
      
# Clean up temporary split PDF files
for i in page_paths:
    os.remove(Path(i))

[PosixPath('/Users/isaaclim/Desktop/OCR/mentis-ocr/cth2_page_1.pdf'), PosixPath('/Users/isaaclim/Desktop/OCR/mentis-ocr/cth2_page_2.pdf'), PosixPath('/Users/isaaclim/Desktop/OCR/mentis-ocr/cth2_page_3.pdf'), PosixPath('/Users/isaaclim/Desktop/OCR/mentis-ocr/cth2_page_4.pdf')]
Output written to /Users/isaaclim/Desktop/OCR/mentis-ocr/extracted_table_1.csv
Invocation details:
- The input length is 332 tokens.
- The output length is 5 tokens.
- The model returned 1 response(s):
1
Output written to /Users/isaaclim/Desktop/OCR/mentis-ocr/extracted_table_2.csv
Invocation details:
- The input length is 343 tokens.
- The output length is 5 tokens.
- The model returned 1 response(s):
0
Output written to /Users/isaaclim/Desktop/OCR/mentis-ocr/extracted_table_3.csv
Invocation details:
- The input length is 324 tokens.
- The output length is 5 tokens.
- The model returned 1 response(s):
3
Output written to /Users/isaaclim/Desktop/OCR/mentis-ocr/extracted_table_4.csv
Invocation details:
- The input 

In [5]:
relevant =  dict(sorted(relevant.items(), key=lambda item: item[1]))
for key, value in relevant.items():
    if value == "1":
        relevant[key] = "ROOM CATEGORY"
    elif value == "2":
         relevant[key] = "MEAL"
    elif value == "3":
         relevant[key] = "TRANSPORT"
    elif value == "4":
         relevant[key] = "EVENT"

In [6]:
for key, value in relevant.items():
     print(f" {key}, {value}")

 /Users/isaaclim/Desktop/OCR/mentis-ocr/extracted_table_1.csv, ROOM CATEGORY
 /Users/isaaclim/Desktop/OCR/mentis-ocr/extracted_table_4.csv, MEAL
 /Users/isaaclim/Desktop/OCR/mentis-ocr/extracted_table_3.csv, TRANSPORT
 /Users/isaaclim/Desktop/OCR/mentis-ocr/extracted_table_5.csv, EVENT


In [7]:
#Final table arrangement

file_path = '/Users/isaaclim/Desktop/OCR/mentis-ocr/final_template_trial.csv'

for key, value in relevant.items():
    df = pd.read_csv(key)

    with open(file_path, 'a') as f:
        f.write('\n')
        f.write(value + '\n')
         
    df.to_csv(file_path, mode='a', header=True, index=False)
    
    # Step 4: Write an empty row after the DataFrame
    with open(file_path, 'a') as f:
        f.write('\n' + " ")